## Importing Data

使用 TensorFlow 一个建议就是使用 `tf.data` 来操作数据。`tf.data` 可以根据简单的可复用片段构建复杂的输入管道。使用 tf.data API 可以轻松处理大量数据、不同的数据格式以及复杂的转换。

`tf.data` API 在 TensorFlow 中引入了两个新的抽象类：

- `tf.data.Dataset` 表示一系列元素，其中每个元素包含一个或多个 Tensor 对象。
- `tf.data.Iterator` 提供了从数据集中提取元素的主要方法。


In [7]:
import numpy as np
import tensorflow as tf

### 基本机制 

要启动输入管道，你必须定义来源。例如，要通过内存中的某些张量构建 `Dataset`, 您可以使用 `tf.data.Dataset.from_tensors()` 或 `tf.data.Dataset.from_tensor_slices()`。

### 数据集结构

一个数据集包含多个元素，每个元素的结构都相同。一个元素包含一个或多个 `tf.Tensor` 对象，这些对象称为组件。每个组件都有一个 `tf.DType`，表示张量中元素的类型；以及一个 `tf.TensorShape`，表示每个元素（可能部分指定）的静态形状。


In [8]:
dataset1 = tf.data.Dataset.from_tensor_slices(
    tf.random.uniform((4, 10))
)
print(dataset1) 

<TensorSliceDataset shapes: (10,), types: tf.float32>


`from_tensor_slices()` 返回一个 Dataset 对象。 Dataset 转换支持任何结构的数据集。在使用 Dataset.map()、Dataset.flat_map() 和 Dataset.filter() 转换时（这些转换会对每个元素应用一个函数），元素结构决定了函数的参数

### 创建迭代器

构建了表示输入数据的 Dataset 后，下一步就是创建 Iterator 来访问该数据集中的元素。tf.data API 目前支持下列迭代器，复杂程度逐渐增大：

- 单次，
- 可初始化，
- 可重新初始化，以及
- 可馈送。

#### 单次

单次迭代器是最简单的迭代器形式，**仅支持对数据集进行一次迭代**，不需要显式初始化。单次迭代器可以处理基于队列的现有输入管道支持的几乎所有情况，但它们不支持参数化。以 Dataset.range() 为例：

In [9]:
dataset = tf.data.Dataset.range(10)
# iterator = dataset.make_one_shot_iterator()
# next_element = iterator.get_next

在 TF2.0 中，TF 的 Dataset 实现了  `__iter__()` 方法，任何实现该方法的对象都成为了一个 iterable 的对象了。那么我就可以更加方便的来使用我们 Dataset 了。

In [10]:
for x in dataset:
    print(x)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


### Dataset 提供的方法

#### shuffle()

注意, 在数据混乱度不够的时候，最好进行下 shuffle, 并且 shuffle 的 size 也足够大，如果小于数据集的大小。那么 shuffle 的效果并不会很好。

[Meaning of buffer_size in Dataset.map , Dataset.prefetch and Dataset.shuffle](https://stackoverflow.com/questions/46444018/meaning-of-buffer-size-in-dataset-map-dataset-prefetch-and-dataset-shuffle)

当 shuffle 的数据混乱度不够，然后进行 batch 那么一个 batch 中的样本可能全是类型一样的样本。那么在训练时就可能不会太好？

问题：假设有 150 个样本，前 [0, 50) 属于 0，[50, 100) 属于 1，[100, 150) 属于 2。 然后分 batch 进行训练，如果不 shuffle 的话，那么训练结果会是怎么样的那？

In [11]:
data = np.arange(10)
dataset = tf.data.Dataset.from_tensor_slices((data))
for x in dataset:
    print(x)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


In [12]:
# 当 buffer_size 为 1 时，是没有 shuffle 的
d_shuffled = dataset.shuffle(1)

for x in d_shuffled:
    print(x)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


In [13]:
# 当 buffer_size 为 5 时，会分成前后两个区域进行 shuffle
d_shuffled = dataset.shuffle(5)

for x in d_shuffled:
    print(x)

tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)


In [15]:
# 一般当数据混乱度不够时 buffer_size 要大于样本个数
# 当然也要考虑内存的问题。
d_shuffled = dataset.shuffle(20)

for x in d_shuffled:
    print(x)

tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)


#### repeat()

对数据进行重复，默认是无限次重复的。

#### batch()

将数据集划分成若干个小的 batch.

In [16]:
data = np.arange(10)
dataset = tf.data.Dataset.from_tensor_slices((data))
for x in dataset:
    print(x)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


In [18]:
for x in dataset.batch(5):
    print(x)   

tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int64)
tf.Tensor([5 6 7 8 9], shape=(5,), dtype=int64)


In [19]:
for x in dataset.batch(3):
    print(x)  

tf.Tensor([0 1 2], shape=(3,), dtype=int64)
tf.Tensor([3 4 5], shape=(3,), dtype=int64)
tf.Tensor([6 7 8], shape=(3,), dtype=int64)
tf.Tensor([9], shape=(1,), dtype=int64)


### slices, 切片

首先，`tf.data.Dataset.from_tensor_slices` 函数创建一个代表数组切片的 `tf.data.Dataset`。系统会在**第一个维度**对该数组进行切片。

**注意**，`Dataset` 是不知道自己包含多少条目的。

例如，一个包含 MNIST 训练数据的数组的形状为 (60000, 28, 28)。将该数组传递给 from_tensor_slices 会返回一个包含 60000 个切片的 Dataset 对象，其中每个切片都是一个 28x28 的图像。


In [20]:
train, test = tf.keras.datasets.mnist.load_data()
mnist_x, mnist_y = train

mnist_ds = tf.data.Dataset.from_tensor_slices(mnist_x)
print(mnist_ds)

<TensorSliceDataset shapes: (28, 28), types: tf.uint8>


### 操作

在对 Dataset 进行替代的时候，会按固定顺序一次产生一个元素。它需要进一步处理才可用于训练。幸运的是，tf.data.Dataset 类提供了更好地准备训练数据的方法。输入函数的下一行就利用了其中的几种方法：


In [23]:
# Shuffle, repeat, and batch the examples.
dataset = dataset.shuffle(1000).repeat().batch(100)
print(dataset)

<BatchDataset shapes: (None, None), types: tf.int64>


- `shuffle` 方法使用一个固定大小的缓冲区，**在条目经过时随机化处理条目**。在这咱情况下，`buffer_size` 大于 `Dataset` 中样本的数量，确保数据完全被随机化处理。

- `repeat()` 方法会在结束时重启 Dataset。要限制周期数量，请设置 `count` 数量。

- `batch()` 方法会收集大量样本并将它们堆叠起来以创建批次。

Dataset 包含 `(features_dict, labels)` 对。这是 `train` 和 `evaluate` 方法的预期格式，因此 `input_fn` 会返回相应的数据集。

使用 `predict` 方法时，可以/应该忽略 `labels`。